In [7]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere_api=Cohere(setting)
print(setting.RERANK)
bert=Bert_Extract(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model
hghaan/rerank_model


## 1.Eval Gemini model

In [2]:
# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
# df = pd.read_csv('./data/data_final_response/final_data_system.csv')

In [3]:
# def Generate_Response(df_batch: pd.DataFrame) -> list[str]:
#     """
#     Sinh câu trả lời cho mỗi câu hỏi trong batch.
#     Trả về danh sách answer tương ứng.
#     """
#     answers = []
#     for question in tqdm(df_batch['question'], desc="Sinh trả lời"):    
#         article_doc = rag.get_gemini_response(question)
#         answers.append(article_doc)
#     return answers

# df['answer_from_gemini'] = Generate_Response(df)
# final_output_path = './data/data_processed/final_data_system_response.csv'
# df.to_csv(final_output_path, index=False)
# print(f" Đã cập nhật và lưu toàn bộ vào: {final_output_path}")


> Configure

In [8]:
setting=Settings()

> Get embedding gemini 

In [9]:
import google.generativeai as genai

genai.configure(api_key=gemini.key_manager.get_next_key())

def get_gemini_embedding(texts):
    try :
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=texts,
            task_type="SEMANTIC_SIMILARITY"
        )
        
        return result['embedding']
    except Exception as e:
        print("Đã xảy ra lỗi:", e)
        return [0.0] * 768

> get embedding cohere 

In [6]:
# import cohere

# def get_cohere_embedding(text: str, model_name: str = "embed-multilingual-v3.0") -> list:
#     try:
#         co = cohere.ClientV2(api_key=cohere_api.key_manager.get_next_key())
#         res = co.embed(
#             texts=[text],
#             model=model_name,
#             input_type="classification",
#             embedding_types=["float"],
#         )a

#         # Trả về nhúng đầu tiên
#         return res.embeddings.float[0]
#     except Exception as e:
#         print("Đã xảy ra lỗi:", e)
#         return [0.0] * 1024  # Trả về danh sách 1024 số 0 nếu có lỗi


>Get  rouge-L & F1 Score

In [10]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

def compute_rouge_l(pred: str, ref: str) -> float:
    try:
        score = scorer.score(ref, pred)
        return score['rougeL'].fmeasure
    except:
        return 0.0

def compute_f1(pred: str, ref: str) -> float:
    pred_tokens = pred.lower().split()
    ref_tokens = ref.lower().split()
    common = set(pred_tokens) & set(ref_tokens)
    if not pred_tokens or not ref_tokens:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * precision * recall / (precision + recall)

In [11]:
import pandas as pd 
df = pd.read_csv('./data/data_processed/final_data_system_response.csv')
df = df.dropna(subset=['answer', 'answer_from_gemini'])

print(df.shape)

(1204, 4)


In [12]:

pattern = r'Xin\s+lỗi\s+bạn'  # regex cho “Xin lỗi bạn” (có thể có nhiều khoảng trắng)

# Chỉ giữ lại những dòng không khớp pattern
df = df[~df['answer_from_gemini_rag_basic'].str.contains(pattern, regex=True, na=False)]

In [17]:
print(df.shape)

(564, 9)


In [14]:
# # df['answer_embedding_cohere'] = df['answer'].apply(get_cohere_embedding)
# df['answer_from_gemini_embedding_cohere'] = df['answer_from_gemini'].apply(get_cohere_embedding)


df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_gemini_embedding'] = df['answer_from_gemini'].apply(get_gemini_embedding)

df['cosine_similarity_gemini'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']], 
        [row['answer_from_gemini_embedding']]
    )[0][0],
    axis=1
)

# df['cosine_similarity_cohere'] = df.apply(
#     lambda row: cosine_similarity(
#         [row['answer_embedding_cohere']], 
#         [row['answer_from_gemini_embedding_cohere']]
#     )[0][0],
#     axis=1
# )

Đã xảy ra lỗi: 400 Request payload size exceeds the limit: 36000 bytes.
Đã xảy ra lỗi: 400 Request payload size exceeds the limit: 36000 bytes.


In [18]:
df['rouge_l'] = df.apply(lambda row: compute_rouge_l(row['answer_from_gemini'], row['answer']), axis=1)
df['f1'] = df.apply(lambda row: compute_f1(row['answer_from_gemini'], row['answer']), axis=1)


In [19]:
print(f" Trung bình Cosine (Gemini): {df['cosine_similarity_gemini'].mean():.4f}")
print(f" Trung bình ROUGE-L: {df['rouge_l'].mean():.4f}")
print(f" Trung bình F1 Score: {df['f1'].mean():.4f}")

 Trung bình Cosine (Gemini): 0.8997
 Trung bình ROUGE-L: 0.3683
 Trung bình F1 Score: 0.2146
